In [5]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../scripts')
import numpy as np
import os, h5py
import pandas as pd
import variant_effect

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# read df and add strand
all_dfs = []
cagi_data = '../data/CAGI'
combined_filename = 'combined_cagi.bed'
for filename in os.listdir(cagi_data):
    prefix, regulator = filename.split('.tsv')[0].split('_')

    one_reg = pd.read_csv(os.path.join(cagi_data,filename), skiprows=7, sep='\t', header=None)
    one_reg['regulator'] = regulator
    one_reg['set'] = prefix
    all_dfs.append(one_reg)
    

combined_cagi = pd.concat(all_dfs)
combined_cagi.insert(4, 'strand', '+')
combined_cagi.insert(2,'end',combined_cagi.iloc[:,1]+1)
combined_cagi.iloc[:,0] = 'chr'+combined_cagi.iloc[:,0].astype(str)
combined_cagi.to_csv(combined_filename, sep='\t', header=False, index=None)

In [10]:
output_filename = 'nonneg_cagi_3K.bed'
variant_effect.expand_range(combined_filename, output_filename)

In [11]:
fa_filename = 'cagi_3k.fa'
coords_list, seqs_list = variant_effect.convert_bed_to_seq(output_filename, fa_filename, genomefile='/home/shush/genomes/hg19.fa')

In [14]:
window = 3072
bad_lines = []
N = len(seqs_list)
nonneg_df = pd.read_csv(output_filename, sep='\t', header=None)
mid = window // 2
onehot_ref = []
onehot_alt = []
coord_np = np.empty((N, 4)) # chrom, start, end coordinate array
pos_dict = {'+': 1535, '-':1536}
for i,(chr_s_e, seq) in enumerate(zip(coords_list, seqs_list)):
    alt = ''
    strand = chr_s_e.split('(')[-1].split(')')[0]
    pos = pos_dict[strand]
#     coord_np[i,3] = pos_dict[strand] - 1535

    if seq[pos] != nonneg_df.iloc[i, 3]:
#         print('Error in line ' + str(i))
        bad_lines.append(i)
    else:
        alt = nonneg_df.iloc[i,4]

        onehot = variant_effect.dna_one_hot(seq)
        mutated_onehot = onehot.copy()
        mutated_onehot[pos] = variant_effect.dna_one_hot(alt)[0]
        onehot_ref.append(onehot)

        onehot_alt.append(mutated_onehot) 

onehot_alt = np.array(onehot_alt)
onehot_ref = np.array(onehot_ref)

In [15]:
included_df = nonneg_df[~nonneg_df.index.isin(bad_lines)]
included_df.to_csv('final_cagi_metadata.csv')

In [16]:
onehot_ref_alt = h5py.File('CAGI_onehot.h5', 'w')
onehot_ref_alt.create_dataset('ref', data=onehot_ref)
onehot_ref_alt.create_dataset('alt', data=onehot_alt)
onehot_ref_alt.close()


## Sanity check that only one nucleotide is different

In [24]:
onehot_ref_alt = h5py.File('CAGI_onehot.h5', 'r')
np.argwhere(onehot_ref_alt['ref'][0,:,:] != onehot_ref_alt['alt'][0,:,:])

array([[1535,    0],
       [1535,    1]])

In [27]:
onehot_ref_alt['ref'][0,1535,:], onehot_ref_alt['alt'][0,1535,:]

(array([0., 1., 0., 0.], dtype=float16),
 array([1., 0., 0., 0.], dtype=float16))